In [2]:
import os
import pickle
import pandas as pd

# bit of loading data
tenders_structured_path = "../../data/UpdatedAgainTenders.xlsx"
tenders_structured = pd.read_excel(tenders_structured_path).astype(str)
tenders_structured = tenders_structured[["Reference Number", "Contract Title"]].dropna(subset=["Reference Number"]).drop_duplicates(subset=["Reference Number"])

summary_map = {}
summary_path = "../../../data/extended_summaries/"
for ref in list(tenders_structured["Reference Number"]):
    sum_file = os.path.join(summary_path, f"{ref}.pickle.sum")
    if os.path.exists(sum_file):
        loaded_sum = pickle.load(open(sum_file, "rb"))
        summary_map[ref] = loaded_sum

In [105]:
from TendersWA import Panda

def compare_summary(ref, display_short_description = False):
    title = tenders_structured.loc[tenders_structured["Reference Number"] == ref, ["Contract Title"]]
    df = pd.DataFrame({"Contract Title": [title], "Source": [summary_map[ref]["relevant"]], "Summary": [summary_map[ref]["summary"]]})
    if display_short_description:
        df["Short Description"] = [summary_map[ref]["short_desc"]]
    Panda.pretty_print(df)

In [64]:
import random

def find_random_tenders(n = 5, min_len = None, max_len = None):
    filter = dict((k, v) for k, v in summary_map.items() if (min_len == None or len(v["relevant"]) > min_len) and (max_len == None or len(v["relevant"]) < max_len) )
    keys = list(filter.keys())
    print(random.choices(keys, k = n))

In [107]:
compare_summary("ED210320", display_short_description = True)

Contract Title,Source,Summary,Short Description
Contract Title 2906 Business Analysis Services for Project Kaartdijin,Business Analysis Services for Project KaartdijinBusiness Analysis Services for Project Kaartdijin,"Business Analysis Services for Project Kaartdijin, a joint venture between the Chinese government and the state-owned oil giant Rosneft.",Business Analysis Services for Project Kaartdijin


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from TendersWA.Models import Embedding_Model as em
model = em.Sentence_transformer()
summ = summary_map["ED210320"]
cosine_similarity([model(summ["relevant"])], [model(summ["summary"]), model(summ["short_desc"])])

array([[0.72754616, 0.9449181 ]], dtype=float32)

In [126]:
compare_summary("ED17105", display_short_description = True)

Contract Title,Source,Summary,Short Description
Contract Title 3441 Provision of Property Valuation Services for the Perth Metropolitan Area,"The Asset Planning and Services Directorate of the Infrastructure Division of the Department requires the specialist services of qualified and experienced Land Valuers for the Perth Metropolitan Area. F Demonstrate an understanding of the Perth Metropolitan area. Travel and Other Additional Costs Travel costs shall not be reimbursed for any valuation services undertaken in the metropolitan area, as defined by the metropolitan scheme boundary. Panel to be Established, it states that it is not mandatory for Panel Members to have knowledge of the Perth metropolitan area. Can you please clarify if it is in fact mandatory for Panel Members to have knowledge of the Perth Metropolitan Area A . It is mandatory for Panel Members to have knowledge of the Perth metropolitan area. It is not mandatory for Panel Members to have knowledge of the Perth metropolitan area. It is mandatory for Panel Members to have knowledge of the Perth metropolitan area. Request REQUEST TITLE Provision of Property Valuation Services for the Perth Metropolitan Area REQUEST NUMBER ED CLOSING TIME PM Friday May , Western Australia ISSUED BY Director General Department of Education Table of Contents PART A REQUEST NO ED . Provision of Property Valuation Services for the Perth Metropolitan AreaProvision of Property Valuation Services for the Perth Metropolitan Area","Provision of Property Valuation Services for the Perth Metropolitan Area is a request from the Department of Education, Western Australia.",Provision of Property Valuation Services for the Perth Metropolitan Area


In [159]:
compare_summary("ERA2019152", display_short_description = True)

Contract Title,Source,Summary,Short Description
Contract Title 13780 Request for Quote (Request) 2019 Container Deposit Scheme Inquiry Non-alcoholic beverage price data,"The Requirement The Western Australian Treasurer has asked the Economic Regulation Authority ERA to monitor and report on the prices of beverages affected by the introduction of the Container Deposit Scheme in Western Australia. A Container Deposit Scheme will be introduced in Western Australia on June . The ERA requires the Contractor to provide data on non alcoholic beverage prices For a broad range of non alcoholic beverages, including soft drinks, water and fruit juice. To understand beverage price changes attributable to the scheme the ERA will utilise difference in differences analysis and will need beverage price data. Your experience with other State Territory container deposit scheme price monitoring if any . The Container Deposit Scheme allows consumers to take empty beverage containers to a refund point to receive a cent refund. This request for quote relates to non alcoholic beverage price data. Part A Statement of Requirement Requirements Overview The Customer is seeking offers for the provision of the non alcoholic beverage price data described in this Request. Request for Quote Request Container Deposit Scheme Inquiry Non alcoholic beverage price data Reference No ERA The Customer is Economic Regulation Authority. To claim the regional content preference and to ensure that the preference is applied correctly where appropriate, businesses must be able to answer the questions set out below Request for Quote Request Container Deposit Scheme Inquiry Non alcoholic beverage price data Container Deposit Scheme Inquiry Non alcoholic beverage price data.",The Western Australian Treasurer has asked the Economic Regulation Authority (ERA) to monitor and report on the prices of beverages affected by the introduction of a Container Deposit Scheme.,Container Deposit Scheme Inquiry Non alcoholic beverage price data.


In [187]:
find_random_tenders(n = 1, min_len = 400, max_len = 700)

['HSS202210094']


In [188]:
compare_summary("HSS202210094", display_short_description = True)

Contract Title,Source,Summary,Short Description
Contract Title 14293 HSS202210094 - HSS Internal Audits Q1 and Q2 2022-23,HSSIA . HSSIA . VAD Application Controls Audit Access Management Logging and Monitoring Controls Review . CUAAFA QUOTE FORM PART A REQUEST FOR QUOTE CUAAFA QUOTE FORM PART B CONTRACTOR OFFER Contractor Representative s Signature CUAAFA QUOTE FORM PART C ACCEPTANCE OF OFFER Delegated Authority s Signature HSS HSS Internal Audits Q and Q To peform two internal audits as part of the approved HSS Internal Audit Plan .,HSS HSS Internal Audits Q and Q To peform two internal audits as part of the approved H SS Internal Audit Plan.,To peform two internal audits as part of the approved HSS Internal Audit Plan . HSSIA . Access Management Logging and Monitoring Controls Review . HSSIA . VAD Application Controls Audit
